In [1]:
# Se importan las librerías a usar.
import pandas as pd
import gzip
import json
from pandas import json_normalize

In [2]:
# Ruta de los archivos a utilizar
ruta_items = "/content/items_corregido.json.gz"
ruta_reviews = "/content/reviews_corregido.json.gz"
ruta_games = "/content/steam_games.json.gz"

In [ ]:
# se crean los dataframes con los que se trabajará.
items = pd.read_json(ruta_items,compression="gzip")

In [52]:
reviews = pd.read_json(ruta_reviews,compression = "gzip")

In [ ]:
games = pd.DataFrame(pd.read_json(ruta_games,compression="gzip",lines=True))

In [4]:
# Se crea una función para guardar los nuevos archivos corregidos

def guardar_archivo(DataFrame,RutaNuevoArchivo):
  ''' Convierte un Dataframe dado en json y lo comprime en formato gzip'''
  archivo_json = DataFrame.to_json()
  with gzip.open(RutaNuevoArchivo, "w") as f:
    f.write(archivo_json.encode('utf-8'))

# Limpieza games

In [ ]:
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [ ]:
# Se eliminan las filas cuyos datos sean todos nulos (ya que no nos proporcionan información sobre el juego).
games.dropna(how = "all", inplace = True)
# Se procede a eliminar las columnas con mayor número de datos incompletos o que se repiten, así como aquellos cuya información no resulta útil para la formulación de las consultas.
games.drop(["title","url","reviews_url","price","early_access","publisher","genres"], axis = 1, inplace = True)


In [ ]:
# Se revisa la base de datos obtenida a partir de la limpieza y se cambian los nombres de las columnas.
games.rename(columns={"app_name":"name","tags":"genre","id":"id_game"},inplace=True)
games

,name,release_date,genre,specs,id_game,developer
88310,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],761140.0,Kotoshiro
88311,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",643980.0,Secret Level SRL
88312,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",670290.0,Poolians.com
88313,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],767400.0,彼岸领域
88314,Log Challenge,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",773570.0,None
...,...,...,...,...,...,...
120440,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",773640.0,"Nikita ""Ghost_RUS"""
120441,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",733530.0,Sacada
120442,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",610660.0,Laush Dmitriy Sergeevich
120443,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",658870.0,"xropi,stev3ns"


In [ ]:
# Se busca la información obtenida hasta el momento.
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         32135 non-null  int64  
 1   name          32133 non-null  object 
 2   release_date  30068 non-null  object 
 3   genre         31972 non-null  object 
 4   specs         31465 non-null  object 
 5   id_game       32133 non-null  float64
 6   developer     28836 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.7+ MB


In [ ]:
#Se reinician los índex
games.reset_index(inplace=True)

In [ ]:
# Se crea un nuevo dataframe para los géneros y para las especificaciones.

games_genres = games[["id_game","genre"]]
games_specs = games[["id_game","specs"]]

# Se eliminan estas columnas del primer data set (ya que ya no son necesarias) así como la columna index.
games.drop(columns=["genre","specs","index"],inplace = True)

In [ ]:
# Se desagregan los datos del género para cada uno de los juegos.
games_genres = games_genres.explode(column=["genre"],ignore_index=True)
games_specs = games_specs.explode(column="specs",ignore_index=True)

In [ ]:
# Se guardan los archivos corregidos en nuevos documentos.
guardar_archivo(games,"/content/games_clean.json.gz")
guardar_archivo(games_genres,"/content/genres_games.json.gz")
guardar_archivo(games_specs,"/content/specs_games.json.gz")

# Limpieza Reviews

In [53]:
# Se visualizan los datos del data frame.
reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25761,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25762,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25763,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25764,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [54]:
# Se elimina la columna "user_url", ya que no será usada.
reviews.drop(columns="user_url",inplace=True)

In [55]:
# Se desagregan cada una de las reviews para cada usuario.
reviews = reviews.explode(column="reviews",ignore_index=True)

In [56]:
reviews

,user_id,reviews
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...
59187,76561198312638244,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59188,76561198312638244,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59189,LydiaMorley,"{'funny': '1 person found this review funny', ..."
59190,LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [57]:
# Con el propósito de separar cada una de las reviews, se normaliza la columna "reviews" la cual se encuentra en formato JSON en un DataFrame nuevo.
x = json_normalize(reviews["reviews"])
# Se combina el DataFrame normalizado con el DataFrame original.
reviews = pd.concat([reviews, x], axis=1).drop('reviews', axis=1)
reviews

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59187,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59188,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59189,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59190,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [65]:
# Se eliminan las columnas que ya no son necesarias o útiles para las consultas que se realizarán.
reviews.drop(columns=["funny","helpful"],inplace=True)

In [70]:
# Se reemplazan los valores no deseados (espacios en blanco) en "last_edited"
reviews["last_edited"] = reviews["last_edited"].replace(r"^\s*$", pd.NA, regex=True)

In [73]:
# Se coincidera que los valores de "last_edited" son los valores de la review más reciente (y por tanto, al año al que pertenecen).
# De esta forma, se decide dejar estos datos, reemplazando aquellos donde se tiene un dato nulo por el valor de la columna "posted"
reviews["last_edited"].fillna(reviews["posted"],inplace=True)

In [75]:
# Se elimina la columna "posted" por redundancia con la columna "last_edited"
reviews.drop(columns="posted",inplace=True)

In [78]:
# Se transforman las fechas en "last_edited" a valores de fecha válidos.


ParserError: ignored

In [76]:
reviews

,user_id,last_edited,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59187,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59188,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59189,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59190,LydiaMorley,Posted July 20.,730,True,:D


In [77]:
reviews.iloc[:25]

,user_id,last_edited,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
5,js41637,"Posted November 29, 2013.",239030,True,Very fun little game to play when your bored o...
6,evcentric,Posted February 3.,248820,True,A suitably punishing roguelike platformer. Wi...
7,evcentric,"Last edited December 5, 2015.",370360,True,'Run for fun? What the hell kind of fun is that?'
8,evcentric,"Posted November 3, 2014.",237930,True,"Elegant integration of gameplay, story, world ..."
9,evcentric,"Posted October 15, 2014.",263360,True,"Random drops and random quests, with stat poin..."
